In [ ]:
import numpy as np
import pandas as pd

from pathlib import Path
import matplotlib.pyplot as plt
import seaborn as sns
from biopsykit.utils.dataframe_handling import multi_xs
import optuna

from biopsykit.stats import StatsPipeline
import pingouin as pg
import biopsykit as bp

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
from fau_colors import register_cmaps
register_cmaps()

sns.set_theme(context="talk", style="white", palette="faculties", font_scale=1.2)

%matplotlib widget

# Correlations to subject informations

In [ ]:
df = pd.read_pickle("full_df.pkl")
df = df.rename(index={"ACT + HRV + EDR":"ACT + HRV + ED-RRV"})

In [ ]:
algorithm = "LSTM"
metric = "accuracy"
stage = "5stage"
modality_list = ["ACT", "ACT + HRV", "ACT + HRV + RRV", "ACT + HRV + ED-RRV"]

# Method to get regression plots

In [ ]:
def get_regression_table(df, metric, algorithm, stage):
    df = multi_xs(df, [metric], level="metric")
    df = multi_xs(df, [algorithm], level="algorithm")
    df = multi_xs(df, [stage], level="stage")
    
    
    df = df[["data", characteristic]].astype(float)
    df_reg_result= df.groupby(["algorithm", "modality", "metric", "stage"]).apply(
        lambda df: pg.linear_regression(X=df[characteristic], y=df["data"])
    )
    df_reg_result = df_reg_result.set_index("names", append=True)
    df_reg_result = df_reg_result.round(3)
    df_reg_result = df_reg_result.xs(characteristic, level="names").sort_values(by="r2", ascending=False).sort_index()
    df_reg_result.sort_index()
    
    return df, df_reg_result.sort_index()

In [ ]:
def get_regression_plot(df, df_reg_result, algorithm, characteristic, characteristic_title):
    algorithm_list = [algorithm]
    df_reg_2 = df_reg_result["r2"]
    df_p = df_reg_result["pval"]
    for alg in algorithm_list:
        fig, ax = plt.subplots(ncols=4, figsize=(25, 7), sharey=True)
        for i, (modality, r2, p) in enumerate(zip(modality_list, df_reg_2, df_p)):

            df_corr = multi_xs(df, [alg], level="algorithm")
            df_corr = multi_xs(df_corr, [modality], level="modality")
            df_corr = multi_xs(df_corr, [stage], level="stage")
            df_corr["data"] = df_corr["data"]
            df_corr[characteristic] = df_corr[characteristic].astype(float)
            df_corr["data"] = df_corr["data"].astype(float)
            df_corr[["data", characteristic]]
            sns.regplot(
                data=df_corr[["data", characteristic]].reset_index(), x=characteristic, y="data", ax=ax[i], line_kws={"color": "#228848"}
            )
            ax[i].set_title(modality + "  r\u00b2 = " + str(r2) + ", p = " + str(p))
            ax[i].set_ylabel("Accuracy [%]")
            ax[i].set_xlabel(characteristic_title)
            ax[i].yaxis.set_tick_params(labelbottom=True)

    plt.tight_layout()
    return fig, ax

# Method to get statstical boxplots for characteristics

In [ ]:
def bin_characteristic(df, characteristic, cutoff, labels = ["bad", "good"]):    
    df["bin_" + characteristic] = pd.cut(
    df[characteristic],
    [df[characteristic].min(), cutoff, df[characteristic].max()],
    right=True,
    include_lowest=True,
    labels=labels
                 )
    return df

In [ ]:
def calculate_stats(df, characteristic):
    df = multi_xs(df, [metric], level="metric")
    df = multi_xs(df, [algorithm], level="algorithm")
    df = multi_xs(df, [stage], level="stage")

    dict_stats = {}

    for modality in modality_list:
        steps = [
            # ("prep", "normality"),
            ("test", "pairwise_tests")
        ]

        params = {
            "dv": "data",
            "between": characteristic,
            "groupby": ["algorithm", "modality", "metric", "stage"],
            "parametric": False,
            "multicomp": {"levels": ["algorithm", "modality"]},
        }

        df_slice = multi_xs(df, [modality], level="modality")
        pipeline = StatsPipeline(steps, params, round=4)
        pipeline.apply(df_slice)

        dict_stats[modality] = pipeline

    return pipeline, dict_stats

In [ ]:
def plot_stats(df, dict_stats, characteristic):
    df = multi_xs(df, [metric], level="metric")
    df = multi_xs(df, [algorithm], level="algorithm")
    df = multi_xs(df, [stage], level="stage")
    
    dict_brackets = {
    key: dict_stats[key].sig_brackets("test",stats_type="between", stats_effect_type="between", plot_type="multi", x="algorithm")
    for key in modality_list
    }
        
    box_pairs = {key: val[0] for key, val in dict_brackets.items()}
    pvalues = {key: val[1] for key, val in dict_brackets.items()}
    
    display(box_pairs)
    display(pvalues)
    
    fig, ax = plt.subplots(figsize=(20, 6), ncols=4)

    features = ["ACT", "ACT + HRV", "ACT + HRV + RRV", "ACT + HRV + ED-RRV"]

    
    fig, ax = bp.plotting.multi_feature_boxplot(
        data=df,
        x="algorithm",
        y="data",
        hue=characteristic,
        group="modality",
        features=features,
        stats_kwargs={
            "box_pairs": box_pairs,
            "pvalues": pvalues,
            "fontsize": "medium",
            "text_offset": -6,
            "verbose": True,
        },
        ylabels={"ACT": "Accuracy [%]", "ACT + HRV": "Accuracy [%]", "ACT + HRV + RRV": "Accuracy [%]", "ACT + HRV + ED-RRV": "Accuracy [%]"},
        axs=ax,
    )
    fig.tight_layout(rect=(0, 0, 0.91, 1), pad=3)
    handles, labels = ax[0].get_legend_handles_labels()

    for a, modality in zip(ax, features):
        a.legend().remove()
        a.set_xlabel("Algorithm")
        a.set_title(modality)
        a.yaxis.set_tick_params(labelbottom=True)

    fig.legends[0].set_title(characteristic)
    
    return fig, ax

In [ ]:
def kruskal_stats(df, characteristic):
    df = multi_xs(df, [metric], level="metric")
    df = multi_xs(df, [algorithm], level="algorithm")
    df = multi_xs(df, [stage], level="stage")
    
    steps = [
    # ("prep", "normality"),
    ("test", "kruskal"),
    ("posthoc", "pairwise_tests"),
]

    params = {
        "dv": "data",
        "between": characteristic,
        "parametric": False,
        "groupby": ["modality", "algorithm", "metric"],
        "multicomp": {"levels": ["modality", "algorithm"]},
    }


    pipeline = StatsPipeline(steps, params, round=4)
    pipeline.apply(df);
    
    return pipeline

# Age

In [ ]:
characteristic = "age"

In [ ]:
df_, reg_result = get_regression_table(df, metric, algorithm, stage)
reg_result

In [ ]:
plt.close("all")
fig, ax = get_regression_plot(df_, reg_result, algorithm, characteristic, characteristic_title="Age")
plt.savefig("Regression_to_age.pdf", format="pdf", bbox_inches="tight")

# AHI

In [ ]:
characteristic = "AH-Index"

In [ ]:
df_, reg_result = get_regression_table(df, metric, algorithm, stage)
reg_result

In [ ]:
plt.close("all")
fig, ax = get_regression_plot(df_, reg_result, algorithm, characteristic, characteristic_title="AH-Index")
plt.savefig("Regression_to_AHI.pdf", format="pdf", bbox_inches="tight")

# WHIIRS_score

In [ ]:
characteristic = "WHIIRS_score"

In [ ]:
df_, reg_result = get_regression_table(df, metric, algorithm, stage)
reg_result

In [ ]:
get_regression_plot(df_, reg_result, algorithm, characteristic, characteristic_title="WHIIRS_score")

# Recording Quality

In [ ]:
characteristic = "PSG_quality"

In [ ]:
df_recording_qual = bin_characteristic(df, characteristic, 5)

In [ ]:
pipeline, dict_stats = calculate_stats(df_recording_qual, "bin_" + characteristic)

In [ ]:
fig, axs = plot_stats(df_recording_qual, dict_stats, "bin_"+characteristic)
fig.legends[0].set_title("PSG Quality")
plt.savefig("recording_quality_psg.pdf", format="pdf", bbox_inches="tight")

# Healthy vs. Sick in general

In [ ]:
df["sick"] = df[["sleep_apnea", "insomnia", "resstles_legs"]].sum(axis=1)
df["sick"] = pd.cut(
    df["sick"], [0, 0.1, 1], include_lowest=True, right=True, labels=["healthy", "sick"]
)

In [ ]:
df

In [ ]:
characteristic = "sick"

In [ ]:
pipeline, dict_stats = calculate_stats(df, characteristic)

In [ ]:
plot_stats(df_recording_qual, dict_stats, characteristic)

# Gender

In [ ]:
characteristic = "gender"

In [ ]:
pipeline, dict_stats = calculate_stats(df, characteristic)

In [ ]:
fig, ax = plot_stats(df_recording_qual, dict_stats, characteristic)
fig.legends[0].set_title("Gender")
plt.savefig("gender_statistics.pdf", format="pdf", bbox_inches="tight")

# Sleep Apnea

In [ ]:
df["sleep_apnea"].replace(to_replace=[0.0, 1.0], value=["healthy", "sick"], inplace=True)
df["insomnia"].replace(to_replace=[0.0, 1.0], value=["healthy", "sick"], inplace=True)
df["resstles_legs"].replace(to_replace=[0.0, 1.0], value=["healthy", "sick"], inplace=True)

In [ ]:
characteristic = "sleep_apnea"

In [ ]:
pipeline, dict_stats = calculate_stats(df, characteristic)

In [ ]:
fig, ax = plot_stats(df_recording_qual, dict_stats, characteristic)
fig.legends[0].set_title("Sleep Apnea")
plt.savefig("sleep_apnea_statistics.pdf", format="pdf", bbox_inches="tight")

# Insomnia

In [ ]:
characteristic = "insomnia"

In [ ]:
pipeline, dict_stats = calculate_stats(df, characteristic)

In [ ]:
fig, ax = plot_stats(df_recording_qual, dict_stats, characteristic)
fig.legends[0].set_title("Insomnia")
plt.savefig("Insomnia_statistics.pdf", format="pdf", bbox_inches="tight")

# Resstles Legs

In [ ]:
characteristic = "resstles_legs"

In [ ]:
pipeline, dict_stats = calculate_stats(df, characteristic)

In [ ]:
fig, ax = plot_stats(df_recording_qual, dict_stats, characteristic)
fig.legends[0].set_title("RLS")
plt.savefig("rls_statistics.pdf", format="pdf", bbox_inches="tight")

# WHIIRS (Insomnia 2)

In [ ]:
characteristic = "WHIIRS_score"

In [ ]:
df_whirrs = bin_characteristic(df, characteristic, 8.9)

In [ ]:
pipeline, dict_stats = calculate_stats(df_recording_qual, "bin_" + characteristic)

In [ ]:
plot_stats(df_recording_qual, dict_stats, "bin_"+characteristic)

# Race

In [ ]:
characteristic = "race"

In [ ]:
pipeline = kruskal_stats(df, characteristic)
pipeline.display_results()

In [ ]:
fig, ax = plt.subplots(ncols=4, figsize=(35,8))

for i, modality in enumerate(modality_list):
    df_race = multi_xs(df, [algorithm], level="algorithm")
    df_race = multi_xs(df_race, [modality], level="modality")
    sns.boxplot(
        data=multi_xs(df, [metric], level="metric").reset_index(),
        x="metric",
        y="data",
        hue="race",
        ax=ax[i],
    )
    ax[i].set_title(modality)
    ax[i].legend().remove()
    ax[i].set_xlabel("")
    ax[i].set_ylabel("Accuracy [%]")


plt.legend(ncol=4, bbox_to_anchor=(-0.5, 1.17))
plt.savefig("origin_statistics.pdf", format="pdf", bbox_inches="tight")
    
